In [ ]:
import requests
import random
import pandas as pd
import time
import math
from copy import deepcopy
from statistics import mean

In [ ]:
def request() -> list:   
    res = requests.get('http://localhost:3000/api/ep1')
    return res.json()

In [ ]:
def data_treatment(json:list):
    for i in range(len(json)):
        for key in json[i]:
            if ('prod' in key):
                if (type(json[i][key])!=dict):
                    qnt = json[i][key]
                    if (type(qnt)==str):
                        qnt = float(qnt)                  
                    if (qnt<0):
                        qnt = 0
                    if (key in [product for product in json[i] if ('prod' in product and (int(product[5]) < 9 and len(product) < 7))]):
                        json[i][key] = {'qnt':[round(qnt)], 'price':None}
                    else:
                        json[i][key] = {'qnt':[qnt], 'price':None}
        try:
            json[i]['date'] = time.ctime(json[i]['date'])
        except:
            continue
    return json

In [ ]:
def generate_random_price(n_min=1.0, n_max=100.0) -> float:
    return round(random.uniform(n_min, n_max), 2)

In [ ]:
def logistic_function(v: float) -> float:
    return 0.5 + (1 / (1 + math.e ** -v))

In [ ]:
def calc_qnt_variation(last_week: list, current_week: list) -> float:
    return mean(current_week) - mean(last_week) / mean(last_week) if (mean(last_week) != 0) else 0

In [ ]:
def calc_new_price(product_db: dict, sales_week: dict) -> dict:
    for product in product_db:
        if (product_db[product]['qnt'] != None):
            if (product not in sales_week.keys()):
                sales_week[product] = [0]
            product_db[product]['next_week_price'] = logistic_function(calc_qnt_variation(product_db[product]['qnt'], sales_week[product])) * product_db[product]['price']
        product_db[product]['qnt'] = sales_week[product]               
    return product_db

In [ ]:
def product_pricing(json: list, product_db: dict) -> tuple:
    sales_week = {}

    for transaction in json:
        for key in transaction:
            if ('prod' in key):
                if (key not in product_db.keys()):
                    transaction[key]['price'] = generate_random_price()
                    sales_week[key] = deepcopy(transaction[key]['qnt'])
                    product_db[key] = deepcopy(transaction[key])
                    product_db[key]['qnt'] = None
                    product_db[key]['next_week_price'] = product_db[key]['price']              
                else:
                    transaction[key]['price'] = deepcopy(product_db[key]['next_week_price'])
                    product_db[key]['price'] = product_db[key]['next_week_price']
                    if (key not in sales_week.keys()):
                        sales_week[key] = deepcopy(transaction[key]['qnt'])
                    else:
                        sales_week[key] += deepcopy(transaction[key]['qnt'])

    product_db = calc_new_price(product_db, sales_week)

    return json, product_db

In [ ]:

def resume (produtos, semana):

   df = produtos.copy()
   somas = []
   prices = []
   colunas = list(df.columns)

   for i in range(df.shape[1]):
      somas.append(sum(df.iloc[0,i]))
      prices.append(df.iloc[1,i])
      
   lista_de_tuplas = list(zip(colunas, somas, prices))
   new_df = pd.DataFrame(lista_de_tuplas, columns = ['produto', 'soma', 'preço'])
   new_df['receita'] = new_df['soma'] * new_df['preço']
   
   new_df['week'] = semana
   
   return new_df
# este código já me dá a receita por semana, mas não por venda

In [ ]:

product_database = {}
weeks = 15
resumo = pd.DataFrame(columns = ['produto', 'soma', 'preço','receita','week'])


for week in range(weeks):
    json = data_treatment(request())
    json, product_database = product_pricing(json, product_database)

    json_db = pd.DataFrame(json)
    product_database_db = pd.DataFrame(product_database)

    print(f'\nSemana {week + 1}:\n')
    print('Json:')
    display(json_db)
    print('\nproduct_database:')
    display(product_database_db)
    semana = 'week' + str(week + 1)
    result_semanal =  resume(product_database_db, semana)
    resumo = resumo.merge(result_semanal, how = 'outer')
    


In [40]:
novo_df = resumo.reset_index()
df_parcial = novo_df.pivot(index = 'week', columns = 'produto', values = 'preço')
df_resultado = df_parcial.fillna(0)
df_resultado
df_resultado.to_csv('weekly_prices.csv')
